<a href="https://colab.research.google.com/github/Mahaveersalvih/movie-recommendation-/blob/main/Copy_of_Project_18_Mood_based_Movie_Recommendation_System_using_Machine_Learning_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the dependencies

In [ ]:
!pip install streamlit
import numpy as np
import pandas as pd
import streamlit as st
import requests
import pickle
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# loading the data from the csv file to apandas dataframe
movies_data = pd.read_csv('/content/movies.csv')

In [ ]:
# printing the first 5 rows of the dataframe
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [ ]:
# number of rows and columns in the data frame

movies_data.shape

(4803, 24)

In [ ]:
# selecting the relevant features for recommendation

selected_features = ['genres','keywords','tagline','cast','director','id']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director', 'id']


In [ ]:
# replacing the null valuess with null string

for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

In [ ]:
# combining all the 5 selected features

combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']+' '+movies_data['id'].astype(str)

In [ ]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [ ]:
# converting the text data to feature vectors

vectorizer = TfidfVectorizer()

In [ ]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [ ]:
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 129068 stored elements and shape (4803, 22079)>
  Coords	Values
  (0, 4962)	0.07558463332496819
  (0, 5035)	0.08675091800727598
  (0, 10035)	0.10682369557117992
  (0, 18360)	0.0996650735240196
  (0, 10198)	0.0996650735240196
  (0, 8439)	0.20571442887524974
  (0, 7826)	0.21356327043425335
  (0, 10597)	0.15835713517009034
  (0, 19139)	0.32659730933843956
  (0, 21348)	0.12067958495774306
  (0, 7986)	0.24002536695991955
  (0, 19032)	0.20571442887524974
  (0, 9706)	0.23104072162345685
  (0, 20022)	0.0682359348921808
  (0, 21759)	0.12329360255434745
  (0, 15953)	0.08702131936522213
  (0, 16264)	0.2616731586704858
  (0, 18110)	0.14444955478119179
  (0, 21768)	0.227362220585816
  (0, 22051)	0.19422995678876445
  (0, 18080)	0.20939292991289057
  (0, 18825)	0.1980589697405917
  (0, 21429)	0.1908195360387445
  (0, 19369)	0.1456939887959868
  (0, 13517)	0.21837757524935328
  :	:
  (4801, 22027)	0.2762414077101103
  (4801, 18596)	0.26675

Cosine Similarity

In [ ]:
# getting the similarity scores using cosine similarity

similarity = cosine_similarity(feature_vectors)

In [ ]:
print(similarity)

[[1.         0.06736808 0.03483192 ... 0.         0.         0.        ]
 [0.06736808 1.         0.03056728 ... 0.03330229 0.         0.        ]
 [0.03483192 0.03056728 1.         ... 0.         0.04952055 0.        ]
 ...
 [0.         0.03330229 0.         ... 1.         0.         0.02447526]
 [0.         0.         0.04952055 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02447526 0.         1.        ]]


In [ ]:
print(similarity.shape)

(4803, 4803)


Getting the movie name from the user

In [ ]:
# getting the movie name from the user

movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name : star


In [ ]:
# creating a list with all the movie names given in the dataset

list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [ ]:
# finding the close match for the movie name given by the user

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Ishtar', 'Superstar', 'Faster']


In [ ]:
close_match = find_close_match[0]
print(close_match)

Ishtar


In [ ]:
# finding the index of the movie with title

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

1256


In [ ]:
# getting a list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, np.float64(0.014169733462846418)), (1, np.float64(0.019574002796500962)), (2, np.float64(0.014471991439824538)), (3, np.float64(0.006765075830240889)), (4, np.float64(0.013824725017162722)), (5, np.float64(0.014638504043251684)), (6, np.float64(0.008145374830677118)), (7, np.float64(0.014292385232656109)), (8, np.float64(0.008298811999017339)), (9, np.float64(0.013988037529846255)), (10, np.float64(0.060206659833057834)), (11, np.float64(0.012688718330902864)), (12, np.float64(0.038455215007198854)), (13, np.float64(0.013137056333237036)), (14, np.float64(0.014084287709247606)), (15, np.float64(0.007716188805642445)), (16, np.float64(0.013604871831888883)), (17, np.float64(0.01386433674678162)), (18, np.float64(0.009954863929858466)), (19, np.float64(0.012404267616756698)), (20, np.float64(0.014115323595707288)), (21, np.float64(0.012026440257824358)), (22, np.float64(0.00732591152147919)), (23, np.float64(0.007032249771961023)), (24, np.float64(0.03573590172380551)), (25, np.floa

In [ ]:
len(similarity_score)

4803

In [ ]:
# sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(1256, np.float64(1.0)), (1477, np.float64(0.17255149199452943)), (1024, np.float64(0.15811327481950238)), (1622, np.float64(0.15395075812542308)), (1607, np.float64(0.13451095031595728)), (2248, np.float64(0.12289277439323978)), (4114, np.float64(0.11196220305660626)), (181, np.float64(0.10777151679791305)), (152, np.float64(0.10495082398142265)), (3083, np.float64(0.10171230544046964)), (137, np.float64(0.09502318257802213)), (985, np.float64(0.09259959901326806)), (272, np.float64(0.0916043326344311)), (3385, np.float64(0.09094392917397554)), (2867, np.float64(0.08881862231086218)), (2045, np.float64(0.08865439586816085)), (2079, np.float64(0.08786808342208915)), (4318, np.float64(0.0870857702056245)), (1812, np.float64(0.08540380593773134)), (1060, np.float64(0.08017417266005852)), (572, np.float64(0.07964569131642062)), (2537, np.float64(0.07939426802657126)), (2679, np.float64(0.0791136136208949)), (1603, np.float64(0.07911264429791451)), (2939, np.float64(0.07836847137839939)),

In [ ]:
# print the name of similar movies based on the index

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Ishtar
2 . Reds
3 . Dick Tracy
4 . Bulworth
5 . Midnight Run
6 . Bon voyage
7 . Subway
8 . Kung Fu Panda
9 . Kung Fu Panda 3
10 . While We're Young
11 . Kung Fu Panda 2
12 . The Messenger: The Story of Joan of Arc
13 . Town & Country
14 . March or Die
15 . Screwed
16 . I Heart Huckabees
17 . Moonlight Mile
18 . Bleeding Hearts
19 . Rain Man
20 . Coyote Ugly
21 . Hook
22 . Wag the Dog
23 . Superstar
24 . Stranger Than Fiction
25 . Quartet
26 . Raise Your Voice
27 . Undiscovered
28 . Tootsie
29 . Gerry


Movie Recommendation Sytem

In [ ]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<20):
    print(i, '.',title_from_index)
    i+=1

 Enter your favourite movie name : superman
Movies suggested for you : 

1 . Superman
2 . Superman II
3 . Superman IV: The Quest for Peace
4 . Man of Steel
5 . Superman III
6 . Crimson Tide
7 . Superman Returns
8 . Suicide Squad
9 . Batman Returns
10 . The Dark Knight Rises
11 . The Killer Inside Me
12 . Nanny McPhee and the Big Bang
13 . Batman Begins
14 . The Dark Knight
15 . The Godfather
16 . Batman
17 . Batman
18 . Batman & Robin
19 . The Island of Dr. Moreau


In [ ]:
import pickle
with open('movie_data.pkl','wb')as file:
    pickle.dump((movies_data,cosine_similarity),file)

In [ ]:
mood_to_genres = {
    'happy': ['Comedy', 'Family', 'Animation', 'Adventure'],
    'sad': ['Drama', 'Tragedy', 'Romance'],
    'action': ['Action', 'Adventure', 'Science Fiction', 'Thriller'],
    'romantic': ['Romance', 'Drama'],
    'thrilling': ['Thriller', 'Action', 'Crime', 'Mystery'],
    'family-friendly': ['Family', 'Animation', 'Comedy', 'Adventure'],
    'dark': ['Thriller', 'Crime', 'Mystery', 'Horror', 'Science Fiction'],
    'funny': ['Comedy', 'Animation', 'Family']
}

print("Mood to genre mapping dictionary created:")
print(mood_to_genres)

Mood to genre mapping dictionary created:
{'happy': ['Comedy', 'Family', 'Animation', 'Adventure'], 'sad': ['Drama', 'Tragedy', 'Romance'], 'action': ['Action', 'Adventure', 'Science Fiction', 'Thriller'], 'romantic': ['Romance', 'Drama'], 'thrilling': ['Thriller', 'Action', 'Crime', 'Mystery'], 'family-friendly': ['Family', 'Animation', 'Comedy', 'Adventure'], 'dark': ['Thriller', 'Crime', 'Mystery', 'Horror', 'Science Fiction'], 'funny': ['Comedy', 'Animation', 'Family']}


In [ ]:
user_mood = input("Enter your current mood (e.g., happy, sad, action, romantic, thrilling, family-friendly, dark, funny): ")
print(f"You entered: {user_mood}")

Enter your current mood (e.g., happy, sad, action, romantic, thrilling, family-friendly, dark, funny): funny
You entered: funny


In [ ]:
genres_for_mood = mood_to_genres.get(user_mood.lower(), [])

if not genres_for_mood:
    print(f"Mood '{user_mood}' not recognized. Please choose from: {', '.join(mood_to_genres.keys())}.")
    print("No genres retrieved for this mood.")
else:
    print(f"Genres associated with '{user_mood}': {genres_for_mood}")

Genres associated with 'funny': ['Comedy', 'Animation', 'Family']


In [ ]:
import pandas as pd
import random

movies_data = pd.read_csv('/content/movies.csv')
movies_data['genres'] = movies_data['genres'].fillna('') # Fill NaN values with empty strings
mood_based_recommendations = []

if genres_for_mood:
    for index, row in movies_data.iterrows():
        movie_genres = row['genres'].split(' ')
        if any(genre in movie_genres for genre in genres_for_mood):
            mood_based_recommendations.append(index)

# Shuffle the recommendations to get different movies each time
random.shuffle(mood_based_recommendations)

print(f"Total movies found for '{user_mood}': {len(mood_based_recommendations)}")
print("First 10 recommended movie indices for this mood (shuffled):")
for i, movie_index in enumerate(mood_based_recommendations[:10]):
    title = movies_data.loc[movie_index, 'title']
    print(f"  {i+1}. {title} (Index: {movie_index})")

Total movies found for 'funny': 1961
First 10 recommended movie indices for this mood (shuffled):
  1. Alfie (Index: 1262)
  2. Space Dogs (Index: 1951)
  3. Big Mommas: Like Father, Like Son (Index: 1505)
  4. Big Momma's House (Index: 1480)
  5. The World's End (Index: 2155)
  6. Chicken Little (Index: 692)
  7. Wish I Was Here (Index: 3482)
  8. Digimon: The Movie (Index: 3613)
  9. Made (Index: 3623)
  10. Diary of a Wimpy Kid (Index: 2525)


In [ ]:
print(f"\nTop 15 Movies for your '{user_mood}' mood:")

if mood_based_recommendations:
    for i, movie_index in enumerate(mood_based_recommendations[:15]):
        title = movies_data.loc[movie_index, 'title']
        print(f"  {i+1}. {title}")
else:
    print("No movies found for this mood.")


Top 15 Movies for your 'funny' mood:
  1. Alfie
  2. Space Dogs
  3. Big Mommas: Like Father, Like Son
  4. Big Momma's House
  5. The World's End
  6. Chicken Little
  7. Wish I Was Here
  8. Digimon: The Movie
  9. Made
  10. Diary of a Wimpy Kid
  11. Super Mario Bros.
  12. The Proposal
  13. Chocolat
  14. All or Nothing
  15. Josie and the Pussycats
